In [3]:
{-# LANGUAGE DataKinds #-}
data TaggedInt   a          = TInt Int
data WrappedInt  a          = WInt (a Int)
data WrappedType a b        = WType (a b)
data IntArgFunc  a          = IArg (Int -> a)
data IntResFunc  a          = IRes (a -> Int)
data KindedInt  (a :: Int)  = KInt Int

Line 2: Use newtype instead of data
Found:
data TaggedInt a = TInt Int
Why not:
newtype TaggedInt a = TInt IntLine 3: Use newtype instead of data
Found:
data WrappedInt a = WInt (a Int)
Why not:
newtype WrappedInt a = WInt (a Int)Line 4: Use newtype instead of data
Found:
data WrappedType a b = WType (a b)
Why not:
newtype WrappedType a b = WType (a b)Line 5: Use newtype instead of data
Found:
data IntArgFunc a = IArg (Int -> a)
Why not:
newtype IntArgFunc a = IArg (Int -> a)Line 6: Use newtype instead of data
Found:
data IntResFunc a = IRes (a -> Int)
Why not:
newtype IntResFunc a = IRes (a -> Int)Line 7: Use newtype instead of data
Found:
data KindedInt (a :: Int) = KInt Int
Why not:
newtype KindedInt (a :: Int) = KInt Int

In [4]:
:k TaggedInt
:k WrappedInt
:k WrappedType
:k IntArgFunc
:k IntResFunc
:k KindedInt
:t fmap

TaggedInt :: forall {k}. k -> *

WrappedInt :: (* -> *) -> *

WrappedType :: forall {k}. (k -> *) -> k -> *

IntArgFunc :: * -> *

IntResFunc :: * -> *

KindedInt :: Int -> *

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

<div style="width: 50%">

- Eine Funktor instanz von TaggedInt ist nicht möglich, da es vom Kind `*` ist
- dasselbe gilt für KindedInt, auch hier ist a ein konkreter wert und kein Typ
- da Wrapped Int vom Kind `(* -> *) -> *` ist, geht auch hier das fmap nicht, da fmap nur EINE Funktion nimmt, hier sind allerdings 2 Funktionen verschachtelt
- ` Expected kind ‘* -> *’, but ‘KindedInt a’ has kind ‘*’` dies zeigt, dass auch KindedInt (genau wie TaggedInt nur ein Wert, nicht aber einen Typ erwartet. Somit ist auch hier keine Instanz möglich)
- für IntResFunc war ich mir sicher, eine Funktor instanz zu finden, leider war dies auch nicht möglich

</div>

```haskell

instance Functor TaggedInt where
    fmap :: (Int -> a) -> TInt Int -> TInt a
    fmap f = \x -> f x

instance Functor a => Functor (WrappedType a) where
    fmap :: (b -> c) -> WrappedType a b -> WrappedType a c
    x :: a b (bound at :8:19)
    f :: b -> c (bound at :8:10)
    fmap f (WType x) = WType (fmap f x)



instance Functor IntResFunc where
    fmap :: (a -> b) -> IntResFunc a -> IntResFunc b
    f :: a -> b
    x :: a -> Int
    n :: b
    fmap f (IRes x) = IRes $ \n -> f $ x n
    fmap f (IRes x) = IRes (f . x)
    fmap f (IRes x) = IRes (\n -> f (x n))

instance Functor (KindedInt a) where
  fmap f (KInt x) = KInt (f x)

In [ ]:
instance Functor IntArgFunc where
    fmap :: (a -> b) -> IntArgFunc a -> IntArgFunc b
    -- f :: a -> b
    -- a :: Int -> a
    fmap f (IArg x) = IArg $ \n -> f $ x n
    -- fmap f (IArg x) = IArg $ f . x

In [ ]:
addOne :: Int -> Int
addOne x = x + 1

maybeInt :: Maybe Int
maybeInt = Just 5

transformedMaybe :: Maybe Int
transformedMaybe = fmap addOne maybeInt

transformedMaybe

Just 6

pure für (-> a)
```haskell
pure' :: a -> f a
        f == ((->) b)


In [ ]:
-- nutzt nicht MyFunctor, damit die Aufgabe unabhängig von der letzten Übung gelöst werden können
class Functor f => MyApplicative f where
  pure' :: a -> f a

  -- entspricht (<*>)
  seqAp' :: f (a -> b) -> f a -> f b

-- nutzt nicht MyApplicative, damit die Teile unabhängig von einander gelöst werden können
class Applicative m => MyMonad m where
  return' :: a -> m a
  return' = pure

  -- entspricht (>>=)
  bind' :: m a -> (a -> m b) -> m b


-- :t (fmap)
-- (>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b
-- (<*>) :: forall (f :: * -> *) a b. Applicative f => f (a -> b) -> f a -> f b
-- (fmap) :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

```haskell
bind' :: m b -> (b -> m c) -> m c
        m == ((-> a))
bind' :: (`-> a` b) -> (b -> (`-> a` c)) -> (`-> a` c)
bind' :: (a -> b)   -> (b -> (a -> c))   -> (a -> c)
bind'       f                   n           a
f :: a -> b
n :: b -> a -> c
a :: a
a => b
b & a => b -> a -> c
1. f a = b
2. n (f a) a

bind' f n a = n (f a) a
bind' f n = \a -> n (f a) a
bind' f = \n a -> n (f a) a
bind' = \f n a -> n (f a) a


```haskell 
bind' :: (m b) -> (b -> m c) -> m c
        m == ((,) a)

bind' :: (((,) a) b) -> (b -> ((,) a) c) -> (((,) a) c)
bind' :: (a , b) -> (b -> (a , c)) -> (a , c)


instance MyMonad ((,) a) where
    bind' :: (a,b) -> (b -> (a,c)) -> (a,c)
    a :: a
    b :: b
    f :: b -> (a,c)
    f b = (a,c)
    bind' (a,b) f = f b
    bind' = \(t1, t2) -> \l -> )

pure :: a -> f a
f == ((,) b)
        a -> (,) b a
        a -> (b,a)

<div style="width: 50%; line-height: 2em">
The pure function solves problems that might occur here (e.g wanting to apply a function that is not residing in the applicative). It accepts a function that is not currently residing in the applicative, and lifts it into the applicative.
<code>pure :: a -> f a</code>
</div>

In [ ]:
instance MyApplicative ((->) a) where
    pure' :: b -> (a -> b)
    -- pure' n = \_ -> n
    -- pure' n _ = n
    pure' = const
    -- seqAp' f g = \x -> f x (g x)
    seqAp' f g x = f x $ g x

    -- mempty ist bestimmt a -> a

instance MyMonad ((->) a) where
    bind' :: (a -> b) -> (b -> (a -> c)) -> (a -> c)
    -- bind' f n = \a -> n (f a) a
    bind' f n a = n (f a) a

instance Monoid a => MyMonad ((,) a) where
    bind' :: (a,b) -> (b -> (a,c)) -> (a,c)
    -- bind' (a,b) f = f b
    bind' (a, b) f = let (a', c) = f b in (a `mappend` a', c)

-- :t (,) :: a -> b -> (a, b)
instance Monoid a => MyApplicative ((,) a) where
    pure' :: b -> (a, b)
    pure' b = (mempty, b)

In [ ]:
test :: (String, Int) -> (Int -> (String, Maybe Int)) -> (String, Maybe Int)
test = bind'

t :: (String, Int)
t = ("2", 2)

m :: Int -> (String, Maybe Int)
m b = (show b, Just b)


test t m 

-- seine : ("22",Just 2)
-- unsere : ("2", Just 2)

("22",Just 2)

In [ ]:
test :: (Int, String) -> (String -> (Int, Maybe String)) -> (Int, Maybe String)
test = bind'

t :: (Int, String)
t = (2, "2")

m :: Int -> (Int, Maybe String)
m b = (b, Just $ show b)


test t m 
-- No instance for (Monoid Int) arising from a use of ‘bind'’

: 